###  ABS to Tidydata

In [1]:
from databaker.framework import *
import pandas as pd 

ModuleNotFoundError: No module named 'databaker'

ABS Excel spreadsheet is available from [ONS website](https://www.ons.gov.uk/businessindustryandtrade/business/businessservices/datasets/annualbusinesssurveyimportersandexporters).

In [ ]:
import requests
from pathlib import Path
from io import BytesIO
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/business/businessservices/datasets/annualbusinesssurveyimportersandexporters/current/importersandexporterssummarytablesinitial.xls'
inputFile = sourceFolder / 'importersandexporterssummarytablesinitial.xls'
response = session.get(inputURL)
with open(inputFile, 'wb') as f:
    f.write(response.content)

In [ ]:
Filenames = [
    'goods_and_services_provisional_2016',
    'goods_provisional_2016',
    'services_provisional_2016',
    'goods_and_services_revised_2015',
    'goods_revised_2015',
    'services_revised_2015',
    'goods_and_services_revised_final_2014',
    'goods_revised_final_2014',
    'services_revised_final_2014'
]

In [ ]:
Filenames[0][-4:]

In [ ]:
tab_main = loadxlstabs(inputFile)

In [ ]:
def user_perc(x):
    
    if x.strip() == 'Number':
        return 'Count'
    else:
        return 'Proportion of all Business'

In [ ]:
def user_perc2(x,y):
    
    if x.strip() == 'Count':
        return int(y)
    else:
        return str(y) 

In [ ]:
def user_perc3(x,y):
    
    if (x.strip() == 'Detailed employment')|(x.strip() == 'Employment'):
        return y
    else:
        return 'Total'

In [ ]:
def user_perc4(x,y):
    
    if (x.strip() == 'Country of Ownership'):
        return y
    else:
        return 'Total'

In [ ]:
def user_perc5(x,y):
    
    if (x.strip() == 'Turnover'):
        return y
    else:
        return 'Total'

In [ ]:
def user_perc6(x,y):
    
    if (x.strip() == 'Age of Business'):
        return y
    else:
        return 'Total'

In [ ]:
def user_perc7(x,y):
    
    if x.strip() == 'Count':
        return y
    else:
        return 'Percent'

In [ ]:
Final_table = pd.DataFrame()
destinationFolder = Path('out')
destinationFolder.mkdir(exist_ok=True, parents=True)

for sh in list(range(0,9)):
    
    tab = tab_main[sh+1]
    observations = tab.excel_ref('B14').expand(RIGHT).expand(DOWN).is_not_blank()- tab.excel_ref('B70').expand(DOWN)
    Trade = tab.excel_ref('B12').expand(RIGHT).is_not_blank()
    CET = tab.excel_ref('A').expand(DOWN).by_index([12,26,36,45,61]).is_not_blank()
    segments = tab.excel_ref('A').expand(DOWN).is_not_blank()- CET - tab.excel_ref('A70').expand(DOWN)-tab.excel_ref('A10').expand(ABOVE)
    measure = tab.excel_ref('B11:F11').expand(RIGHT).is_not_blank()
    Dimensions = [
                HDimConst('Geography', 'K02000001'),
                HDimConst('Year', Filenames[sh][-4:]),
                HDim(CET,'CET',CLOSEST,ABOVE),
                HDim(measure,'Measure Type', CLOSEST, LEFT), 
                HDimConst('Unit', 'Businesses'),
                HDim(segments,'Statistics', DIRECTLY, LEFT ),
                HDim(Trade,'Export and Import Activity',DIRECTLY,ABOVE)

    ] 
    c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
    new_table = c1.topandas()
    new_table['Export and Import Activity'] = new_table['Export and Import Activity'].str.rstrip('47')
    new_table['CET'] = new_table['CET'].str.rstrip('34')
    new_table['Measure Type'] = new_table['Measure Type'].str.rstrip('56')
    new_table['Measure Type'] = new_table['Measure Type'].map(lambda cell:cell.replace('of', ''))
    new_table['Measure Type'] = new_table['Measure Type'].map(lambda cell:cell.replace('%', 'Percentage'))
    new_table['Statistics'] = new_table['Statistics'].astype(str)
    new_table['Statistics'] = new_table['Statistics'].map(lambda cell:cell.replace('+', ' Above'))
    new_table['CET'] = new_table['CET'].str.replace('3','')
    new_table = new_table.drop('DATAMARKER', axis=1)
    new_table.rename(index= str, columns= {'OBS':'Value'}, inplace = True)
    new_table = new_table[new_table['Value'] != '']
    new_table['Year'] = new_table['Year'].apply(str)
    new_table['Measure Type'] = new_table.apply(lambda row: user_perc(row['Measure Type']), axis = 1)
    new_table['Value'] = new_table.apply(lambda row: user_perc2(row['Measure Type'], row['Value']), axis = 1)
    new_table['Unit'] = new_table.apply(lambda row: user_perc7(row['Measure Type'], row['Unit']), axis = 1)
    new_table['CET'] = new_table['CET'].map(lambda cell:cell.replace("Turnover   £'000s", 'Turnover'))
    new_table['CET'] = new_table['CET'].map(lambda cell:cell.replace('Age (years) ', 'Age of Business'))                                                                                                                           
    new_table['Employment'] = new_table.apply(lambda row: user_perc3(row['CET'], row['Statistics']), axis = 1)
    new_table['Country of Ownership'] = new_table.apply(lambda row: user_perc4(row['CET'], row['Statistics']), axis = 1)
    new_table['Turnover'] = new_table.apply(lambda row: user_perc5(row['CET'], row['Statistics']), axis = 1)
    new_table['Age of Business'] = new_table.apply(lambda row: user_perc6(row['CET'], row['Statistics']), axis = 1)
    new_table =new_table[['Geography','Year','Export and Import Activity','Employment','Turnover','Age of Business','Country of Ownership','Measure Type','Value','Unit']]
    Final_table = pd.concat([Final_table, new_table])
    Final_table.to_csv(destinationFolder / ('ABS.csv'), index = False)


In [ ]:
Final_table.head(5)

In [ ]:
Final_table.tail(5)